## SoS Workflow:

This is the options and the SoS code to run the LDSC pipeline using your own data. 

## Command Interface:

In [1]:
!sos run LDSC_Code.ipynb -h

usage: sos run LDSC_Code.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  make_annot
  munge_sumstats_no_sign
  munge_sumstats_sign
  calc_ld_score
  convert_ld_snps
  calc_enrichment

Sections
  make_annot:
    Workflow Options:
      --bed VAL (as str, required)
                        path to bed file
      --bim VAL (as str, required)
                        path to bim file
      --annot VAL (as str, required)
                        name of output annotation file
  munge_sumstats_no_sign:
    Workflow Options:
      --sumst VAL (as str, required)
                        path to summary statistic file
      --alleles 'w_hm3.snplist'
                        path to Hapmap3 SN

## Make Annotation File:

In [93]:

[make_annot]

# Make Annotated Bed File

# path to bed file
parameter: bed = str 
#path to bim file
parameter: bim = str
#name of output annotation file
parameter: annot = str
bash: 
    python2 /mnt/mfs/statgen/Anmol/ldsc/make_annot.py --bed-file {bed} --bimfile {bim} --annot-file {annot}

## Munge Summary Statistics (Option 1: No Signed Summary Statistic File)

In [ ]:
# Option when Summary Statistic File does not contain a Z or Beta Column (Signed Summary Statistic)

[munge_sumstats_no_sign]



#path to summary statistic file
parameter: sumst = str
#path to Hapmap3 SNPs file, keep all columns (SNP, A1, and A2) for the munge_sumstats program
parameter: alleles = "w_hm3.snplist"
#path to output file
parameter: output_sumst = str
#does summary statistic contain Z or Beta
parameter: signed = False

bash: expand = '${ }'
   if [${signed}==True]
       then
           python2 /mnt/mfs/statgen/Anmol/ldsc/munge_sumstats.py --sumstats ${sumst} --merge-alleles ${alleles} --out ${output_sumst} --a1-inc
       fi

## Munge Summary Statistics (Option 2: Signed Summary Statistic File)

In [ ]:
# This option is for when the summary statistic file does contain a signed summary statistic (Z or Beta)
[munge_sumstats_sign]



#path to summary statistic file
parameter: sumst = str
#path to Hapmap3 SNPs file, keep all columns (SNP, A1, and A2) for the munge_sumstats program
parameter: alleles = "w_hm3.snplist"
#path to output file
parameter: output_sumst_2 = str
#does summary statistic contain Z or Beta
parameter: signed = False

bash: expand = '${ }'
    if [${signed}==False]
        then
            python2 /mnt/mfs/statgen/Anmol/ldsc/munge_sumstats.py --sumstats ${sumst} --merge-alleles ${alleles} --out ${output_sumst_2}
        fi

## Calculate LD Scores

In [ ]:
[calc_ld_score]

#Path to directory with bim files
parameter: bim = path()
#Path to directory with annotation files, output will appear here too. Make sure to remove the SNP, CHR, and BP columns from the annotation files if present before running.
parameter: annot_files = path()
#number of features
parameter: num_features = int

bash: expand = '${ }'
   #echo {annot_files} > out.txt
   seq 1 ${num_features}| xargs -n 1 -I j -P 4 python2 /mnt/mfs/statgen/Anmol/ldsc/ldsc.py --bfile ${bim}/1000G.EUR.QC.22 --l2 --ld-wind-cm 1 --annot ${annot_files}/feat_j_chr_22.annot.gz --thin-annot --out ${annot_files}/feat_j_chr_22 --print-snps /mnt/mfs/statgen/Anmol/ldsc/tutorial_data/w_hm3.snplist/snplist.txt
   seq 1 22| xargs -n 1 -I j -P 4 python2 /mnt/mfs/statgen/Anmol/ldsc/ldsc.py --bfile ${bim}/1000G.EUR.QC.j --l2 --ld-wind-cm 1 --annot ${annot_files}/base_chr_j.annot.gz --thin-annot --out ${annot_files}/base_chr_j --print-snps /mnt/mfs/statgen/Anmol/ldsc/tutorial_data/w_hm3.snplist/snplist.txt

## Create Separate LD Score Files for AD Summary Statistic SNP Format

In [ ]:
# Convert SNP format in LD Score Files to CHR:BP to match with AD Summary Statistic Format


[convert_ld_snps]

#Path to directory with ld score files AND annotation files
parameter: ld_scores = str

parameter: num_features = int


R: expand = "${ }", container="/mnt/mfs/statgen/Anmol/r-packages.sif"
    library(tidyverse)
    #library(R.utils)
    library(data.table)
    for (i in seq(1,22)){
      data = read.table(gzfile(paste0("${ld_scores}/base_chr_",i,".l2.ldscore.gz")),header=T)
      data_2 = fread(paste0("${ld_scores}/base_chr_",i,".l2.M_5_50"))
      data_3 = read.table(gzfile(paste0("${ld_scores}/base_chr_",i,".annot.gz")),header=T)
      data$SNP = paste0(data$CHR,":",data$BP)
      fwrite(data,paste0("${ld_scores}/AD_base_chr_",i,".l2.ldscore.gz"),quote=F,sep="\t",row.names=F,col.names=T)
      fwrite(data_2,paste0("${ld_scores}/AD_base_chr_",i,".l2.M_5_50"),quote=F,sep="\t",row.names=F,col.names=F)
      fwrite(data_3,paste0("${ld_scores}/AD_base_chr_",i,".annot.gz"),quote=F,sep="\t",row.names=F,col.names=T)
      for (j in seq(1,${num_features})){
      data = read.table(gzfile(paste0("${ld_scores}/feat_",j,"_chr_",i,".l2.ldscore.gz")),header=T)
      data_2 = fread(paste0("${ld_scores}/feat_",j,"_chr_",i,".l2.M_5_50"))
      data_3 = read.table(gzfile(paste0("${ld_scores}/feat_",j,"_chr_",i,".annot.gz")),header=T)
      data$SNP = paste0(data$CHR,":",data$BP)
      fwrite(data,paste0("${ld_scores}/AD_feat_",j,"_chr_",i,".l2.ldscore.gz"),quote=F,sep="\t",row.names=F,col.names=T)
      fwrite(data_2,paste0("${ld_scores}/AD_feat_",j,"_chr_",i,".l2.M_5_50"),quote=F,sep="\t",row.names=F,col.names=F)
      fwrite(data_3,paste0("${ld_scores}/AD_feat_",j,"_chr_",i,".annot.gz"),quote=F,sep="\t",row.names=F,col.names=T)
    }
    }
  


## Calculate Enrichments

In [ ]:
[calc_enrichment]

#Path to Control Summary statistics File
parameter: ctrl_sumstats = str
#Path to AD Summary statistics File
parameter: AD_sumstats = str
#Path to Reference LD Scores File Directory 
parameter: ref_ld = str
#Path to LD Weight Files for Control Sumstats (Format like minimal working example)
parameter: w_ld_ctrl = str
#path to frequency files for Control Sumstats (Format like minimal working example)
parameter: frq_file_ctrl = str
#Path to LD Weight Files for AD Sumstats (Format like minimal working example)
parameter: w_ld_AD = str
#path to frequency files for AD Sumstats (Format like minimal working example)
parameter: frq_file_AD = str
#Number of Features
parameter: num_features = int 
#Control Phenotype, For Output
parameter: pheno = str

bash: expand = '${ }'
    seq 1 ${num_features}| xargs -n 1 -I j -P 4 python2 /mnt/mfs/statgen/Anmol/ldsc/ldsc.py --h2 ${ctrl_sumstats} --ref-ld-chr ${ref_ld}/base_chr_,${ref_ld}/feat_j_chr_ --w-ld-chr ${w_ld_ctrl} --overlap-annot --frqfile-chr ${frq_file_ctrl} --out ${ref_ld}/${pheno}_feat_j
    seq 1 ${num_features}| xargs -n 1 -I j -P 4 python2 /mnt/mfs/statgen/Anmol/ldsc/ldsc.py --h2 ${AD_sumstats} --ref-ld-chr ${ref_ld}/AD_base_chr_,${ref_ld}/AD_feat_j_chr_ --w-ld-chr ${w_ld_AD} --overlap-annot --frqfile-chr ${frq_file_AD} --out ${ref_ld}/AD_feat_j